In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import tensorflow as tf
from tensorflow import keras
import tensorboard
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import random
import os
import sklearn
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,AveragePooling2D,BatchNormalization,GlobalAveragePooling2D

In [4]:
data_aug_train=ImageDataGenerator(rescale=1.0/255,
                            rotation_range=30,
                            shear_range=0.2,
                            zoom_range=0.15,
                           horizontal_flip=True,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            fill_mode='nearest')

In [5]:
data_aug_test=ImageDataGenerator(rescale=1.0/255)

In [6]:
train_gen=data_aug_train.flow_from_directory('train',
                                       target_size=(48,48),
                                       batch_size=16,
                                       class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [7]:
test_gen=data_aug_test.flow_from_directory('test',
                                          target_size=(48,48),
                                          batch_size=16,
                                          class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [8]:

model=Sequential([
                  Conv2D(16,kernel_size=(3,3),padding='same',input_shape=(48,48,3),activation='leaky_relu', kernel_initializer=keras.initializers.glorot_normal),
                  BatchNormalization(),
                  Conv2D(16,kernel_size=(3,3),padding='same',activation='leaky_relu',kernel_initializer=keras.initializers.glorot_normal),
                  BatchNormalization(),
                  
                  Conv2D(32,kernel_size=(3,3),padding='same',activation='leaky_relu',kernel_initializer=keras.initializers.glorot_normal),
                  BatchNormalization(),
                  Conv2D(32,kernel_size=(3,3),padding='same',activation='leaky_relu',kernel_initializer=keras.initializers.glorot_normal),
                  BatchNormalization(),
                  MaxPooling2D((2,2)),
                  Dropout(rate=0.10),
                  
                  Conv2D(48,kernel_size=(3,3),padding='same',activation='leaky_relu',kernel_initializer=keras.initializers.glorot_normal),
                  BatchNormalization(),
                  Conv2D(48,kernel_size=(3,3),padding='same',activation='leaky_relu',kernel_initializer=keras.initializers.glorot_normal),
                  BatchNormalization(),
                  MaxPooling2D((2,2)),
                  Dropout(rate=0.10),
    
                  Conv2D(64,kernel_size=(3,3),padding='same',activation='leaky_relu',kernel_initializer=keras.initializers.glorot_normal),
                  GlobalAveragePooling2D(),
                  
                  Flatten(),
                  Dense(7,activation='softmax')
                 ])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 16)        448       
                                                                 
 batch_normalization (Batch  (None, 48, 48, 16)        64        
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 16)        2320      
                                                                 
 batch_normalization_1 (Bat  (None, 48, 48, 16)        64        
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 32)        4640      
                                                                 
 batch_normalization_2 (Bat  (None, 48, 48, 32)        1

In [10]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
batch_size=16

In [12]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir="./model2", histogram_freq=1, write_graph=True, write_images=True,update_freq="epoch")

In [14]:
model.fit(train_gen,
          steps_per_epoch=28709//batch_size,
          validation_data=test_gen,
          validation_steps=7178//batch_size,
          epochs=32,
          callbacks=[tbCallBack])

Epoch 1/32
1794/1794 [==============================] - 376s 209ms/step - loss: 1.7699 - accuracy: 0.2664 - val_loss: 1.8483 - val_accuracy: 0.2794
Epoch 2/32
1794/1794 [==============================] - 501s 279ms/step - loss: 1.6543 - accuracy: 0.3363 - val_loss: 1.6423 - val_accuracy: 0.3553
Epoch 3/32
1794/1794 [==============================] - 507s 282ms/step - loss: 1.5557 - accuracy: 0.3903 - val_loss: 1.5259 - val_accuracy: 0.4348
Epoch 4/32
1794/1794 [==============================] - 502s 280ms/step - loss: 1.4630 - accuracy: 0.4385 - val_loss: 1.4492 - val_accuracy: 0.4819
Epoch 5/32
1794/1794 [==============================] - 509s 284ms/step - loss: 1.3999 - accuracy: 0.4647 - val_loss: 1.3431 - val_accuracy: 0.4893
Epoch 6/32
1794/1794 [==============================] - 517s 288ms/step - loss: 1.3645 - accuracy: 0.4818 - val_loss: 1.4499 - val_accuracy: 0.4870
Epoch 7/32
1794/1794 [==============================] - 510s 284ms/step - loss: 1.3296 - accuracy: 0.4965 - val_

In [119]:
path="test/"
filelist=os.listdir(path)
test_sec = 0
count_sec = [0] * 7
true_sec = np.zeros((7,3))
Acc = np.zeros((7,3))
pred_len = 50
for folder in filelist:
    test_sec += 1
    print('sec')
    content_list = []
    content_id = os.listdir(path+folder+'/')
    for i in range(pred_len):
        random_content = random.choice(content_id)
        content_list.append(random_content)
 
    for item in content_list:
        count_sec[test_sec-1] += 1
        img=cv2.imread(path+folder+'/'+item)
        img=cv2.resize(img,(48,48))
        img=np.reshape(img,(1,48,48,3))
        img = img / 255
        pred_prob = model.predict(img)
        for N in range(1,4):
            N_max = np.argsort(pred_prob[0])[-N:]
            if test_sec - 1 in N_max:
                true_sec[test_sec-1][N-1] += 1

sec
1/1 [==============================] - 0s 22ms/step
sec
1/1 [==============================] - 0s 34ms/step
sec
1/1 [==============================] - 0s 26ms/step
sec
1/1 [==============================] - 0s 32ms/step
sec
1/1 [==============================] - 0s 28ms/step
sec
1/1 [==============================] - 0s 22ms/step
sec
1/1 [==============================] - 0s 23ms/step


In [123]:
Acc = true_sec /pred_len
Avg = sum(Acc) / 7
print(Avg)
print(Acc)
print(emotions)

[0.53142857 0.71142857 0.82285714]
[[0.66 0.78 0.86]
 [0.2  0.42 0.7 ]
 [0.26 0.54 0.68]
 [0.84 0.9  0.9 ]
 [0.66 0.82 0.9 ]
 [0.5  0.8  0.94]
 [0.6  0.72 0.78]]
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
